# Retrieval-Augmented Generation (RAG) System

## Problem Statement

Large Language Models (LLMs) generate responses based on their pre-trained knowledge, which may be outdated or inaccurate. To improve response accuracy, a Retrieval-Augmented Generation (RAG) system is used.

This project builds a RAG pipeline that retrieves relevant information from a document and provides it to a language model to generate context-aware and accurate answers.

## Objectives

- Load a document as a knowledge source
- Split the document into smaller text chunks
- Convert text into vector embeddings
- Store embeddings in a vector database
- Retrieve relevant chunks based on user queries
- Generate accurate responses using a language model

## Dataset / Knowledge Source

- **Data Type:** PDF  
- **File Name:** attention-is-all-you-need.pdf
- **Domain:** attention-is-all-you-need(tranformer mechanism)

This document serves as the knowledge base for the RAG system. The model retrieves relevant information from this PDF to answer user queries.

**File Path:**  
/content/drive/MyDrive/RAG_Pipeline_Agentic_AI(ASSIGNMENT1)/attention-is-all-you-need.pdf

In [2]:
import os

file_path = "/content/drive/MyDrive/RAG_Pipeline_Agentic_AI(ASSIGNMENT1)/attention-is-all-you-need.pdf"

if os.path.exists(file_path):
    print("Dataset uploaded successfully.")
else:
    print("File not found. Please upload ml_intro.pdf")

Dataset uploaded successfully.


## Step 1: Install Required Libraries

The following libraries are required for building the RAG system:

- **LangChain** – Framework for building RAG pipelines  
- **FAISS** – Vector database for similarity search  
- **PyPDF** – PDF document loader  
- **Sentence Transformers** – Embedding generation  
- **Transformers** – Language model support  

These libraries are installed using pip in Google Colab.

## Step 2: Import Libraries

In this step, all required modules are imported for:

- Loading the PDF document
- Splitting text into chunks
- Generating embeddings
- Creating a vector database (FAISS)
- Building the Retrieval-Augmented Generation pipeline
- Running a language model for answer generation



This project uses lightweight libraries to build a stable RAG pipeline:

- PyPDF – PDF loading
- Sentence Transformers – Text embeddings
- FAISS – Vector similarity search
- Transformers – Language model for answer generation

In [1]:
# =============================
# INSTALL DEPENDENCIES (RUN ONCE PER SESSION)
# =============================
!pip install -q "numpy<2.0" faiss-cpu pypdf sentence-transformers transformers

# =============================
# MOUNT GOOGLE DRIVE
# =============================
from google.colab import drive
drive.mount('/content/drive')

# =============================
# IMPORT LIBRARIES
# =============================
import os
import numpy as np
import faiss
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import pipeline

print("Environment Ready ✅")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Environment Ready ✅


## Step 3: Load PDF Document

The PDF document is loaded from Google Drive using PyPDF.

Each page of the document is extracted as text and stored for further processing.

**File Path:**  
/content/drive/MyDrive/ml_intro.pdf

In [3]:
file_path = "/content/drive/MyDrive/RAG_Pipeline_Agentic_AI(ASSIGNMENT1)/attention-is-all-you-need.pdf"

reader = PdfReader(file_path)

documents = []
for page in reader.pages:
    text = page.extract_text()
    if text:
        documents.append(text)

print("Total pages loaded:", len(documents))
print("Sample text preview:\n")
print(documents[0][:500])

Total pages loaded: 15
Sample text preview:

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz 


## Step 4: Text Chunking Strategy

The extracted text is divided into smaller chunks to improve retrieval performance.

**Chunk Size:** 500 characters  
**Chunk Overlap:** 50 characters  

### Reason for Selection
- Maintains context continuity between chunks
- Prevents loss of important information at chunk boundaries
- Improves semantic search accuracy
- Balances retrieval quality and memory efficiency

In [5]:
chunk_size = 500
chunk_overlap = 50

chunks = []

for doc in documents:
    start = 0
    while start < len(doc):
        end = start + chunk_size
        chunk = doc[start:end]
        chunks.append(chunk)
        start = end - chunk_overlap

print("Total chunks created:", len(chunks))
print("\nSample chunk:\n")
print(chunks[0])

Total chunks created: 96

Sample chunk:

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.edu
Łukasz 


## Step 5: Embedding Generation

Text chunks are converted into numerical vector representations (embeddings) for semantic similarity search.

**Embedding Model Used:**  
sentence-transformers/all-MiniLM-L6-v2

### Reason for Selection
- Lightweight and fast
- Good semantic understanding
- Suitable for CPU execution
- Open-source and free
- Widely used for semantic search tasks

In [23]:
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Embedder ready")

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Embedder ready


In [24]:
embeddings = model.encode(chunks, show_progress_bar=True)

embeddings = np.array(embeddings)

print("Embedding shape:", embeddings.shape)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Embedding shape: (96, 384)


## RAG Architecture

User Query  
↓  
Query Embedding  
↓  
FAISS Vector Search  
↓  
Retrieve Top Relevant Chunks  
↓  
Combine Context + Query  
↓  
Language Model  
↓  
Generated Answer

## Step 6: Vector Database

The generated embeddings are stored in a vector database for efficient similarity search.

**Vector Store Used:** FAISS (Facebook AI Similarity Search)

### Reason for Selection
- Fast nearest-neighbor search
- Efficient for large vector datasets
- Works locally without external services
- Lightweight and easy to integrate

In [25]:
dimension = embeddings.shape[1]

index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print("Total vectors stored in FAISS:", index.ntotal)

Total vectors stored in FAISS: 96


In [27]:
def retrieve(query, top_k=3):
    query_vector = embedder.encode([query])
    D, I = index.search(query_vector, top_k)
    return [chunks[i] for i in I[0]]


## Step 7: Language Model Setup

A language model is used to generate answers based on the retrieved document context.

**Model Used:** google/flan-t5-base

### Reason for Selection
- Open-source and free
- Optimized for question-answering and instruction tasks
- Lightweight enough to run in Google Colab
- Produces structured and relevant responses

In [10]:
from transformers import pipeline

generator = pipeline(
    "text-generation",   # Updated task
    model="google/flan-t5-base",
    max_length=256
)

print("LLM loaded successfully")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Passing `generation_config` together with generation-related arguments=({'max_length'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'AfmoeForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'CwmForCausalLM', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCa

LLM loaded successfully


## Step 8: Build RAG Pipeline

The Retrieval-Augmented Generation process consists of:

1. User enters a query
2. Query is converted into an embedding
3. FAISS retrieves the most relevant text chunks
4. Retrieved chunks are combined as context
5. Context + query are given to the language model
6. The model generates the final answer

This ensures responses are grounded in the document instead of relying only on model knowledge.

In [17]:
def rag_answer(query, top_k=3):

    # Retrieve relevant chunks from paper
    retrieved_chunks = retrieve(query, top_k=top_k)


    # Combine context
    context = "\n".join(retrieved_chunks)

    # Paper-specific prompt
    prompt = f"""
You are an AI assistant answering questions about the research paper
"Attention Is All You Need".

Use ONLY the context below to answer.
If answer is not present, say "Not found in paper."

Context:
{context}

Question:
{query}

Answer:
"""

    # Generate response (controlled length)
    result = generator(prompt, max_length=150)[0]["generated_text"]

    return result.strip()


## Step 9: Testing the RAG System

The system is tested using multiple queries to evaluate its ability to retrieve relevant information and generate accurate responses.

Minimum three queries are tested as required.

In [18]:
test_queries = [
    "What is the Transformer architecture?",
    "Why does the paper remove recurrence?",
    "What is self-attention?",
    "What are the main components of the Transformer?",
    "What problem does the paper solve?"
]


In [28]:
for i, query in enumerate(test_queries, 1):
    print(f"\nQuery {i}: {query}")
    answer = rag_answer(query)
    print("Answer:")
    print(answer)
    print("-" * 80)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Query 1: What is the Transformer architecture?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer:
You are an AI assistant answering questions about the research paper
"Attention Is All You Need".

Use ONLY the context below to answer.
If answer is not present, say "Not found in paper."

Context:
Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1 Encoder and Decoder Stacks
Encoder: The encoder is composed of a stack of N = 6 identical layers. Each layer has two
sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-
wise fully conn
ting point operations used to train a
model by multiplying the training time, the number of GPUs used, and an estimate of the sustained
single-precision floating-point capacity of each GPU 5.
6.2 Model Variations
To evaluate the importance of different components of the Transformer, we varie

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer:
You are an AI assistant answering questions about the research paper
"Attention Is All You Need".

Use ONLY the context below to answer.
If answer is not present, say "Not found in paper."

Context:
es an output
sequence (y1, ..., ym) of symbols one element at a time. At each step the model is auto-regressive
[10], consuming the previously generated symbols as additional input when generating the next.
2
2) O(1) O(logk(n))
Self-Attention (restricted) O(r · n · d) O(1) O(n/r)
3.5 Positional Encoding
Since our model contains no recurrence and no convolution, in order for the model to make use of the
order of the sequence, we must inject some information about the relative or absolute position of the
tokens in the sequence. To this end, we add "positional encodings" to the input embeddings at the
bottoms of the encoder and decoder stacks. The positional encodings have the same dimension dmodel
as t
ithout regard to their distance in
the input or output sequences [2, 19]. In all bu

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer:
You are an AI assistant answering questions about the research paper
"Attention Is All You Need".

Use ONLY the context below to answer.
If answer is not present, say "Not found in paper."

Context:
teract with Multi-Head Attention as
described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehension, abstractive summarization,
textual entailment and learning task-independent sentence representations [4, 27, 28, 22].
End-to-end memory networks are based on a re
le of the attention mechanism following long-distance dependencies in the
encoder self-attention in layer 5 of 6. Many of the attention heads attend to a distant dependency of
the verb ‘making’, completing the phrase ‘making...more difficult’. Attentions here shown only for
the word ‘making’

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer:
You are an AI assistant answering questions about the research paper
"Attention Is All You Need".

Use ONLY the context below to answer.
If answer is not present, say "Not found in paper."

Context:
Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1 Encoder and Decoder Stacks
Encoder: The encoder is composed of a stack of N = 6 identical layers. Each layer has two
sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-
wise fully conn
ting point operations used to train a
model by multiplying the training time, the number of GPUs used, and an estimate of the sustained
single-precision floating-point capacity of each GPU 5.
6.2 Model Variations
To evaluate the importance of different components of the Transformer, we varie

In [3]:
!pip install -q gradio


In [4]:
import gradio as gr

def ask_pdf(question):
    if not question.strip():
        return "Please enter a question."

    answer = rag_answer(question)
    return answer


demo = gr.Interface(
    fn=ask_pdf,
    inputs=gr.Textbox(
        placeholder="Ask something about the paper...",
        label="Your Question"
    ),
    outputs=gr.Markdown(label="Answer"),
    title="📄 RAG PDF Question Answering",
    description="Ask questions about the paper: Attention Is All You Need",
    theme="soft"
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1919caaae5592221ec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Step 10: Future Improvements

The current RAG system is functional but can be enhanced in several ways:

### Possible Enhancements
- Implement semantic or dynamic chunking instead of fixed-size chunking
- Use hybrid search (keyword + semantic search)
- Add reranking models to improve retrieval accuracy
- Implement metadata-based filtering for multi-document search
- Store FAISS index persistently instead of rebuilding each time
- Support multiple file formats (PDF, TXT, DOCX, Web)
- Develop a Streamlit or Gradio interface for interactive use

## Project Overview

This project implements a Retrieval-Augmented Generation (RAG) system that answers user queries using information retrieved from a PDF document. The system improves response accuracy by grounding the language model in relevant document context.

## Tools and Libraries Used

- Python
- Google Colab
- PyPDF
- Sentence Transformers
- FAISS (Vector Database)
- HuggingFace Transformers

## RAG Workflow

User Query  
→ Query Embedding  
→ FAISS Similarity Search  
→ Retrieve Relevant Chunks  
→ Context + Query  
→ Language Model  
→ Generated Answer

## Instructions to Run

1. Open the notebook in Google Colab
2. Upload the PDF file to Google Drive
3. Update the file path if necessary
4. Run all cells sequentially
5. Modify the query section to test custom questions